# **Pyspark Groupby and Aggregate Function**

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Create the Pyspark Session**

In [3]:
from pyspark.sql import SparkSession

**Let's create a Session App Name**

In [4]:
spark = SparkSession.builder.appName('Pyspark_Machine_Learning').getOrCreate()

**Let's Check the session name**

In [5]:
spark

**Read the dataframe using spark**

In [6]:
df = spark.read.option('header', 'true').csv('test1.csv', inferSchema=True)

In [7]:
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



**Check the column names**

In [8]:
df.columns

['Name', 'age', 'Experience', 'Salary']

In [9]:
from pyspark.ml.feature import VectorAssembler
feature_assembler = VectorAssembler(inputCols=['age', 'Experience'], outputCol='Independent_cols')

In [10]:
df_output = feature_assembler.transform(df)

In [11]:
df_output.show()

+---------+---+----------+------+----------------+
|     Name|age|Experience|Salary|Independent_cols|
+---------+---+----------+------+----------------+
|    Krish| 31|        10| 30000|     [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|      [30.0,8.0]|
|    Sunny| 29|         4| 20000|      [29.0,4.0]|
|     Paul| 24|         3| 20000|      [24.0,3.0]|
|   Harsha| 21|         1| 15000|      [21.0,1.0]|
|  Shubham| 23|         2| 18000|      [23.0,2.0]|
+---------+---+----------+------+----------------+



In [12]:
df_output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent_cols']

In [13]:
final_df = df_output.select(['Independent_cols', 'Salary'])

In [14]:
final_df.show()

+----------------+------+
|Independent_cols|Salary|
+----------------+------+
|     [31.0,10.0]| 30000|
|      [30.0,8.0]| 25000|
|      [29.0,4.0]| 20000|
|      [24.0,3.0]| 20000|
|      [21.0,1.0]| 15000|
|      [23.0,2.0]| 18000|
+----------------+------+



In [46]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = final_df.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol = 'Independent_cols', labelCol='Salary')
regressor = regressor.fit(train_data)

In [47]:
regressor.coefficients

DenseVector([28.4757, 1271.3568])

In [48]:
regressor.intercept

14299.83249581293

In [49]:
pred_results = regressor.evaluate(test_data)

In [50]:
pred_results.predictions.show()

+----------------+------+------------------+
|Independent_cols|Salary|        prediction|
+----------------+------+------------------+
|     [31.0,10.0]| 30000|27896.147403685143|
+----------------+------+------------------+



In [53]:
pred_results.meanSquaredError, pred_results.meanAbsoluteError

(4426195.747020763, 2103.852596314857)